In [1]:
"""
Unit tests for as_list function.
"""
None

In [2]:
%run "./as_list"

In [3]:
import unittest
from operator import itemgetter
import pyspark.sql.types as pst

class Test_as_list(unittest.TestCase):
  
  def test_deep_converts_fields(self):
    input = [ pst.Row(id=1, a=pst.Row(b=101)) ]
    input_df = spark.createDataFrame(input)

    actual = as_list(input_df, True)
    expect = [{'id':1, 'a':{'b':101}}]

    self.assertEqual(actual, expect)

  def test_shallow_doesnt_convert_fields(self):
    input = [ pst.Row(id=1, a=pst.Row(b=101)) ]
    input_df = spark.createDataFrame(input)

    actual = as_list(input_df, False)
    expect = [{'id':1, 'a':pst.Row(b=101)}]

    self.assertEqual(actual, expect)
    
  def test_unsorted_doesnt_match(self):
    """We expect that a DataFrame would not match
    a list equality if not having the same items order."""
    input = [(9, 'Zztop'),
             (1, 'John')]
    input_df = spark.createDataFrame(input, ['id', 'name'])
    
    expect = [(1, 'John'),
              (9, 'Zztop')]
    
    actual = as_list(input_df)
    self.assertNotEqual(actual, expect)
    
  def test_sorted_match(self):
    input = [(9, 'Zztop'),
             (1, 'John')]
    input_df = spark.createDataFrame(input, ['id', 'name'])
    
    expect = [(1, 'John'),
              (9, 'Zztop')]
    
    actual = as_list(input_df, order_by=itemgetter('id'))
    self.assertNotEqual(actual, expect)
    


In [4]:
# Use unittest to execute test cases  
suite = unittest.TestLoader().loadTestsFromTestCase(Test_as_list)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

test_deep_converts_fields (__main__.Test_as_list) ... ok
test_shallow_doesnt_convert_fields (__main__.Test_as_list) ... ok
test_sorted_match (__main__.Test_as_list) ... ok
test_unsorted_doesnt_match (__main__.Test_as_list)
We expect that a DataFrame would not match ... ok

----------------------------------------------------------------------
Ran 4 tests in 1.794s

OK
Out[4]: <unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [5]:
help(as_list)

Help on function as_list in module __main__:

as_list(input_df: pyspark.sql.dataframe.DataFrame, deep=False, order_by=None)
 Convert Spark DataFrame to Python list of dictionaries.
 
 The DataFrame is collected and each row is converted into
 a dictionary. Optionally fields of pyspark Row type are 
 also recursively converted to dictionary.
 
 The resulting list could be sorted, using a supplied key function.
 Sorted list is useful, for example, to compare against expcted 
 fixture.
 
 Parameters:
 input_df (pyspark.sql.DataFrame): Spark DataFrame to be converted.
 
 deep (bool): (Default=False) Whether Row to dictionary conversion should be deep.
 
 order_by (callable): (Default=None) Key function to order by. If provided, the result 
 list will be sorted using the provided key function.